# Explore the dataset

In [1]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import numpy as np
import os, sys
import argparse

import torch
import torch.nn as nn
import torch.optim as optim

from model import HierarchicalAttentionNetwork
from dt import News20Dataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups
#from transformers import DistilBertForSequenceClassification, Trainer, TrainingArgument
from transformers import BertConfig
from transformers import BertForSequenceClassification

from transformers import AdamW
from torch.utils.data import DataLoader
import torch.nn.functional as F
from utils import get_pretrained_weights, Tokenizor

In [2]:
# the size of training dataset
raw_data = fetch_20newsgroups(
    data_home='data/news20',
    subset='train',
    categories=['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space'],
    shuffle=False,
    remove=('headers', 'footers', 'quotes'))

In [3]:
X, y = raw_data['data'], raw_data['target']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42) 
maximum_vocab_size = 10000
min_count = 3
tokenizor = Tokenizor(X, maximum_vocab_size, min_freq=min_count)
train_dataset = News20Dataset(
    X_train, y_train, tokenizor, 200,40)
val_dataset = News20Dataset(
    X_test, y_test, tokenizor, 200, 40)


In [4]:
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
#dev = torch.device('cpu')

In [6]:

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))
            


def get_preprocess(dev):
    def preprocess(*batch_data):
        return [x.to(torch.int64).to(dev) for x in batch_data]
    
    return preprocess
preprocess = get_preprocess(dev)
val_dl = MyDataLoader(val_dataset, batch_size=32)
train_dl = MyDataLoader(train_dataset, batch_size=32)

val_dl = WrappedDataLoader(val_dl, preprocess)
train_dl = WrappedDataLoader(train_dl, preprocess)


In [ ]:
            self.optimizer.zero_grad()

            loss = self.criterion(scores, labels)
            loss.backward()

            if self.config.max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(
                    self.model.parameters(), self.config.max_grad_norm)

            # NOTE MODIFICATION (BUG): clip grad norm should come before optimizer.step()
            self.optimizer.step()

            # Compute accuracy
            predictions = scores.max(dim=1)[1]
            correct_predictions = torch.eq(predictions, labels).sum().item()
            acc = correct_predictions

            self.losses.update(loss.item(), batch_size)
            self.accs.update(acc, batch_size)

In [ ]:
num_classes=4
vocab_size=tokenizor.vocab_size
embed_dim=300
word_gru_hidden_dim=256
sent_gru_hidden_dim=256
word_gru_num_layers=1
sent_gru_num_layers=1
word_att_dim=64
sent_att_dim=64
use_layer_norm=True
dropout=0.5

In [18]:
model = HierarchicalAttentionNetwork(
    num_classes=4,
    vocab_size=tokenizor.vocab_size+10,
    embed_dim=300,
    word_gru_hidden_dim=256,
    sent_gru_hidden_dim=256,
    word_gru_num_layers=1,
    sent_gru_num_layers=1,
    word_att_dim=64,
    sent_att_dim=64,
    use_layer_norm=True,
    dropout=0.5).to(dev)

In [9]:
#self, batch_size, output_size, hidden_size, vocab_size, embedding_length, weights):
model = Fasttext(output_size= train_dataset.num_classes,
            vocab_size=train_dataset.vocab_size,
            embedding_length=300,
            dropout=0.2#self, batch_size, output_size, hidden_size, vocab_size, 
)
from utils import get_pretrained_weights
model.word_embeddings.weights = get_pretrained_weights("data/glove", train_dataset.vocab, 300, dev)
model.train()
model.to(dev)

NameError: name 'Fasttext' is not defined

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [20]:
max_grad_norm = None
for epoch in range(15):
    print(epoch,'--------------------------------------------')
    for i, (docs, labels, doc_lengths, sent_lengths) in enumerate(train_dl):
        model.train()
        scores, word_att_weights, sentence_att_weights = model(
                docs, doc_lengths, sent_lengths)
        loss = F.cross_entropy(scores, labels)
        loss.backward()

        if max_grad_norm is not None:
            torch.nn.utils.clip_grad_norm_(
                self.model.parameters(), max_grad_norm)


        optimizer.step()
        optimizer.zero_grad()

        if (i + len(train_dl) * epoch) % 50==0:
            print('iterate %d loss %.2f' %(i + len(train_dl) * epoch, loss))
   
    model_eval(model, val_dl)

0 --------------------------------------------
iterate 0 loss 1.40
iterate 50 loss 1.28
precision 0.3455056179775281
1 --------------------------------------------
iterate 100 loss 1.26
precision 0.47331460674157305
2 --------------------------------------------
iterate 150 loss 0.78
precision 0.6643258426966292
3 --------------------------------------------
iterate 200 loss 0.41
precision 0.7331460674157303
4 --------------------------------------------
iterate 250 loss 0.53
precision 0.7205056179775281
5 --------------------------------------------
iterate 300 loss 0.28
precision 0.7528089887640449
6 --------------------------------------------
iterate 350 loss 0.27
precision 0.7682584269662921
7 --------------------------------------------
iterate 400 loss 0.05
precision 0.7570224719101124
8 --------------------------------------------
iterate 450 loss 0.04
precision 0.7780898876404494
9 --------------------------------------------
iterate 500 loss 0.11
precision 0.7724719101123596


In [26]:
model.eval()
print()

In [27]:

# the size of training dataset
raw_data = fetch_20newsgroups(
    data_home='data/news20',
    subset='test',
    categories=['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space'],
    shuffle=False,
    remove=('headers', 'footers', 'quotes'))
X, y = raw_data['data'], raw_data['target']
test_dataset = News20Dataset(X_test, y_test, "data/glove/glove.6B.300d.txt", 200) 

test_dl = DataLoader(test_dataset, batch_size=32)
test_dl = WrappedDataLoader(test_dl, preprocess)

model_eval(model, test_dl)


precision 0.7542134831460674
